In [1]:
# some magic so that the notebook will reload external python modules;
# see https://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
import json
import random
import os
import numpy as np

In [3]:
label_filepath = "/ssd_scratch/cvit/vanshg/phrases_dataset/akshat_phrases/labels.txt"
label_dir = os.path.dirname(label_filepath)
print(label_dir)

/ssd_scratch/cvit/vanshg/phrases_dataset/akshat_phrases


In [4]:
label_lines = open(label_filepath, 'r').readlines()

print(len(label_lines))

250


In [6]:
label_lines = open(label_filepath, 'r').readlines()

# rng = random.Random(42)
# rng.shuffle(label_lines)
print(label_lines[0])

akshat_phrases/processed_videos/1720866143961_0_6692555fbc1718e22960f6ea.mp4 I NEED HELP GETTING UP PLEASE



In [7]:
label_lines = open(label_filepath, 'r').readlines()
rng = random.Random(42)

N = len(label_lines)
test_size = 0.3
train_label_filepath = os.path.join(label_dir, "train_labels.txt")
test_label_filepath = os.path.join(label_dir, "test_labels.txt")

num_train = 200
num_test = 50

# rng.shuffle(label_lines)

# Creating the Train label file
with open(train_label_filepath, 'w') as file:
    for i in range(num_train):
        print(f"{i}, {label_lines[i]}", end='')
        file.write(label_lines[i])

# Creating the Test label file
with open(test_label_filepath, 'w') as file:
    for i in range(num_train, N):
        print(f"{i}, {label_lines[i]}", end='')
        file.write(label_lines[i])

0, akshat_phrases/processed_videos/1720866143961_0_6692555fbc1718e22960f6ea.mp4 I NEED HELP GETTING UP PLEASE
1, akshat_phrases/processed_videos/1720866178398_1_66925582bc1718e22960f6f9.mp4 CAN YOU BRING ME SOME WATER
2, akshat_phrases/processed_videos/1720866196056_2_66925594bc1718e22960f70c.mp4 HOW ARE YOU FEELING TODAY
3, akshat_phrases/processed_videos/1720866213073_3_669255a5bc1718e22960f723.mp4 WHAT WOULD YOU LIKE FOR DINNER
4, akshat_phrases/processed_videos/1720866246963_4_669255c6bc1718e22960f73e.mp4 COULD YOU CALL THE DOCTOR FOR ME
5, akshat_phrases/processed_videos/1720866263447_5_669255d7bc1718e22960f75d.mp4 I'M HAVING TROUBLE BREATHING
6, akshat_phrases/processed_videos/1720866272634_6_669255e0bc1718e22960f780.mp4 CAN YOU HELP ME TO THE BATHROOM
7, akshat_phrases/processed_videos/1720866282294_7_669255eabc1718e22960f7a7.mp4 MY ARM HURTS A LOT TODAY
8, akshat_phrases/processed_videos/1720866298514_8_669255fabc1718e22960f7d2.mp4 LET'S TALK ABOUT YOUR DAY AT SCHOOL
9, akshat_